In [2]:
!pip install openai-whisper keybert nltk vaderSentiment soundfile librosa --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 13.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 6.5 MB/s eta 0:00:00


In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import zipfile
import os

# Paths
archive_path = '/content/drive/MyDrive/UserLibri/archive3.zip'  # your zip file
extract_path = '/content/drive/MyDrive/UserLibri/archive3_unzipped/'  # folder to unzip into

# Unzip if not already unzipped
if not os.path.exists(extract_path):
    with zipfile.ZipFile(archive_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print(f"Archive extracted to: {extract_path}")
else:
    print(f"Archive already extracted at: {extract_path}")


Archive already extracted at: /content/drive/MyDrive/UserLibri/archive3_unzipped/


In [5]:
# ============================================
# Full Processing Pipeline with Empathy/Pro/Compliance
# ============================================

import os
import pandas as pd
import whisper
import nltk
from keybert import KeyBERT
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# -------------------------------
# Step 1: Download NLTK Data
# -------------------------------
nltk.download('stopwords')
nltk.download('punkt')

# -------------------------------
# Step 2: Set dataset path
# -------------------------------
dataset_path = '/content/drive/MyDrive/UserLibri/archive3_unzipped/UserLibri/audio_data/'

# -------------------------------
# Step 3: Collect all audio files
# -------------------------------
audio_files = []
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.endswith('.flac') or file.endswith('.mp3') or file.endswith('.wav'):
            audio_files.append(os.path.join(root, file))

print(f"Total audio files found: {len(audio_files)}")

# Optional: test only first 5 files
audio_files = audio_files[:5]

# -------------------------------
# Step 4: Load Whisper model
# -------------------------------
model = whisper.load_model("base")  # "base" = fast; "small"/"medium" = more accurate

# -------------------------------
# Step 5: Initialize NLP tools
# -------------------------------
kw_model = KeyBERT()
analyzer = SentimentIntensityAnalyzer()

# -------------------------------
# Step 6: Text cleaning function
# -------------------------------
filler_words = ['uh', 'um', 'ah', 'erm', 'hmm']

def clean_text(text):
    text = text.lower()
    for filler in filler_words:
        text = text.replace(filler, '')
    text = ' '.join(text.split())
    return text

# -------------------------------
# Step 7: Empathy / Professionalism / Compliance functions
# -------------------------------
def empathy_score(text):
    empathy_words = [
        "sorry", "understand", "help", "assist", "apologize",
        "appreciate", "concern", "thank you", "happy", "glad"
    ]
    count = sum(text.lower().count(w) for w in empathy_words)
    return round(min(1.0, count / 3.0), 2)

def professionalism_score(text):
    bad_words = ["damn", "shit", "fuck", "stupid", "idiot"]
    slang_words = ["lol", "wtf", "bro", "dude", "ain't"]

    score = 1.0
    if any(w in text.lower() for w in bad_words):
        score -= 0.4
    if any(w in text.lower() for w in slang_words):
        score -= 0.3
    if len(text.split()) < 3:
        score -= 0.2

    return round(max(0.0, score), 2)

def compliance_flag(text):
    sensitive_items = [
        "credit card", "account number", "guarantee",
        "refund", "confidential", "private", "email", "dob"
    ]
    return 1 if any(s in text.lower() for s in sensitive_items) else 0

# -------------------------------
# Step 8: Process each audio file
# -------------------------------
all_data = []

for audio_file in audio_files:
    print(f"\nProcessing: {audio_file}")

    # 8a: Transcription
    result = model.transcribe(audio_file)
    transcript = result['text']

    # 8b: Clean / normalize text
    cleaned_text = clean_text(transcript)

    # 8c: Keyword extraction
    keywords = kw_model.extract_keywords(cleaned_text, top_n=5)
    keywords_list = [k[0] for k in keywords]

    # 8d: Sentiment analysis
    sentiment_score = analyzer.polarity_scores(cleaned_text)['compound']
    if sentiment_score >= 0.05:
        sentiment = 'Positive'
    elif sentiment_score <= -0.05:
        sentiment = 'Negative'
    else:
        sentiment = 'Neutral'

    # 8e: Special scoring
    empathy = empathy_score(cleaned_text)
    professionalism = professionalism_score(cleaned_text)
    compliance = compliance_flag(cleaned_text)

    # 8f: Print output in real-time
    print("Clean Text:", cleaned_text)
    print("Keywords:", keywords_list)
    print("Sentiment:", sentiment)
    print("Empathy:", empathy)
    print("Professionalism:", professionalism)
    print("Compliance:", compliance)

    # 8g: Save results to list
    all_data.append({
        'Audio_File': os.path.basename(audio_file),
        'Clean_Text': cleaned_text,
        'Keywords': keywords_list,
        'Sentiment': sentiment,
        'Empathy': empathy,
        'Professionalism': professionalism,
        'Compliance': compliance
    })

# -------------------------------
# Step 9: Save all results to CSV
# -------------------------------
df = pd.DataFrame(all_data)
output_path = '/content/drive/MyDrive/UserLibri_Processed_withScores.csv'
df.to_csv(output_path, index=False)
print(f"\nProcessing complete! Output saved to: {output_path}")

# -------------------------------
# Optional: Show first 5 rows
# -------------------------------
df.head()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Total audio files found: 4050

Processing: /content/drive/MyDrive/UserLibri/archive3_unzipped/UserLibri/audio_data/test-clean/speaker-1089-book-4217/1089-134686-0000.flac


/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Clean Text: he hoped there would be stew for dinner, turnips and carrots and bruised potatoes and fat mutton pieces to be ladled out in thick, peppered flour-fatten sauce.
Keywords: ['stew', 'potatoes', 'dinner', 'carrots', 'mutton']
Sentiment: Positive
Empathy: 0.0
Professionalism: 1.0
Compliance: 0

Processing: /content/drive/MyDrive/UserLibri/archive3_unzipped/UserLibri/audio_data/test-clean/speaker-1089-book-4217/1089-134686-0001.flac


/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


...

/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Clean Text: hello, bertie, any good in your mind?
Keywords: ['bertie', 'hello', 'good', 'mind']
Sentiment: Positive
Empathy: 0.0
Professionalism: 1.0
Compliance: 0

Processing: /content/drive/MyDrive/UserLibri/archive3_unzipped/UserLibri/audio_data/test-clean/speaker-1089-book-4217/1089-134686-0004.flac


/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Clean Text: nber 10, fresh nelly is waiting on you. good night, husband.
Keywords: ['nelly', 'nber', 'fresh', 'husband', '10']
Sentiment: Positive
Empathy: 0.0
Professionalism: 1.0
Compliance: 0

Processing complete! Output saved to: /content/drive/MyDrive/UserLibri_Processed_withScores.csv


,Audio_File,Clean_Text,Keywords,Sentiment,Empathy,Professionalism,Compliance
0,1089-134686-0000.flac,"he hoped there would be stew for dinner, turni...","[stew, potatoes, dinner, carrots, mutton]",Positive,0.0,1.0,0
1,1089-134686-0001.flac,"stuffed into you, his belly countled him.","[stuffed, belly, countled]",Neutral,0.0,1.0,0
2,1089-134686-0002.flac,"after early nightfall, the yellow lamps would ...","[brothels, lamps, nightfall, squalid, yellow]",Neutral,0.0,0.7,0
3,1089-134686-0003.flac,"hello, bertie, any good in your mind?","[bertie, hello, good, mind]",Positive,0.0,1.0,0
4,1089-134686-0004.flac,"nber 10, fresh nelly is waiting on you. good n...","[nelly, nber, fresh, husband, 10]",Positive,0.0,1.0,0
